This notebook is a "meta-notebook" in that it generates other notebooks to actually get run.

The substitution rules for template notebooks are:

* The template must start with "TEMPLATE", which will be removed in the output name.
* The template should follow papermill's parametrization rules
* TARGETNAME and SLITNAME will be changed if they appear in the template's name

In [1]:
import re
from pathlib import Path

import yaml

import papermill

from astroquery.mast import Observations

In [2]:
PROPOSAL_ID = 2609

In [3]:
template_paths = list(Path('.').glob('TEMPLATE*.ipynb'))
template_paths

[PosixPath('TEMPLATETARGETNAME_SLITNAME_slit_dl_and_pipeline.ipynb')]

In [4]:
for path in template_paths:
    params = papermill.inspect_notebook(template_paths[0])
    assert 'PROPOSAL_ID' in params
    assert 'TARGET_NAME' in params
    assert 'SLIT_NAME' in params

    print(path)
    print(params)

TEMPLATETARGETNAME_SLITNAME_slit_dl_and_pipeline.ipynb
{'DL_PATH_NAME': {'name': 'DL_PATH_NAME', 'inferred_type_name': 'None', 'default': "'mastDownload/flat/'", 'help': ''}, 'SLIT_NAME': {'name': 'SLIT_NAME', 'inferred_type_name': 'None', 'default': "'s00066'", 'help': ''}, 'PROPOSAL_ID': {'name': 'PROPOSAL_ID', 'inferred_type_name': 'None', 'default': '2609', 'help': ''}, 'REDUX_BASE_NAME': {'name': 'REDUX_BASE_NAME', 'inferred_type_name': 'None', 'default': "'miscdata/jwst_outputs/'", 'help': ''}, 'TARGET_NAME': {'name': 'TARGET_NAME', 'inferred_type_name': 'None', 'default': "'NGC6791-NEWCATALOG-GAIAEDR3'", 'help': 'If None, matches against the whole program'}, 'CONTENT_DIR': {'name': 'CONTENT_DIR', 'inferred_type_name': 'None', 'default': "'/containerapp/content'", 'help': ''}}


# Select the slits to run

In [5]:
obses = Observations.query_criteria(proposal_id=PROPOSAL_ID)
specobses = obses[(obses['calib_level']==3)&(obses['dataproduct_type'] == 'spectrum')]
specobses

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str7,str12,str4,str12,str8,str27,str23,str40,float64,float64,str8,str14,int64,float64,float64,float64,float64,float64,str60,float64,str4,str2,int64,str115,str66,str67,str16,bool,float64,str9,str9
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s01035_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.074579882 61.877969446 28.075552087 61.877969446 28.075552087 61.877667174 28.074579882 61.877667174,mast:JWST/product/jw02609-o007_s01035_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s01035_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544641,390754414
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s00970_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.067236965 61.887788335 28.068213199 61.887788335 28.068213199 61.887483777 28.067236965 61.887483777,mast:JWST/product/jw02609-o007_s00970_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s00970_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544655,390754416
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s06046_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.188633601 61.87839925 28.189616714 61.87839925 28.189616714 61.87808739 28.188633601 61.87808739,mast:JWST/product/jw02609-o007_s06046_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s06046_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544665,390754417
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s01040_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.080246846 61.872897263 28.081216217 61.872897263 28.081216217 61.872596172 28.080246846 61.872596172,mast:JWST/product/jw02609-o007_s01040_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s01040_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544677,390754418
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s05379_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.204804994 61.888792613 28.205787251 61.888792613 28.205787251 61.888479405 28.204804994 61.888479405,mast:JWST/product/jw02609-o007_s05379_nirspec_f100lp-g140h_cal.jpg,mast:JWST/product/jw02609-o007_s05379_nirspec_f100lp-g140h_s2d.fits,PUBLIC,False,nan,100544688,390754420
science,JWST,CALJWST,NIRSPEC/MSA,JWST,F100LP;G140H,Infrared,IC166-NEWCATALOG-GAIAEDR3,--,jw02609-o007_s05185_nirspec_f100lp-g140h,28.09702,61.86512777777778,spectrum,"Nidever, David",3,59891.81792711713,59891.855679699074,291.778,810.0,5000.0,Searching for the Alpha-Abundance Bimodality in the M31 Disk,60256.96119201,2609,GO,--,POLYGON 28.169943836 61.862103826 28.170927537 61

In [6]:
target_name = 'NGC6791-NEWCATALOG-GAIAEDR3'
specnames = []
for row in specobses[specobses['target_name'] == target_name]:
    m = re.match('jw.*_(s.*?)_.*', row['obs_id'])
    if m is not None:
        specnames.append((target_name, m.group(1)))

# Generate the parametrization notebook files

In [7]:
outbase = Path('miscdata/jwst_outputs') 
outbase.mkdir(exist_ok=True)

outputs = []
for targetname, slitname in specnames:
    for tpath in template_paths:
        targetpath = outbase / tpath.with_suffix('.yaml').name.removeprefix('TEMPLATE').replace('SLITNAME', slitname).replace('TARGETNAME', targetname)
        print('Writing', targetpath)
        with targetpath.open('w') as f:
            yaml.dump({'DL_PATH_NAME': 'mastDownload/flat/',
                        'REDUX_BASE_NAME': 'miscdata/jwst_outputs/',
                        'PROPOSAL_ID': str(PROPOSAL_ID),
                        'SLIT_NAME': slitname,
                        'TARGET_NAME': targetname,
                       }, f)
        outpath = targetpath.with_suffix('.ipynb')
        outputs.append((tpath, targetpath, outpath))

Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s00066_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s07401_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01449_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01782_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s07918_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01421_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01451_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01462_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s08149_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01742_slit_dl_and_pipeline.yaml
Writing miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01327_slit_dl_and_pipeline.yaml

# Commands to execute the notebooks

In [8]:
for tpath, yamlpath, outpath in outputs:
    print(f'papermill {tpath} {outpath} -f {yamlpath}')

papermill TEMPLATETARGETNAME_SLITNAME_slit_dl_and_pipeline.ipynb miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s00066_slit_dl_and_pipeline.ipynb -f miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s00066_slit_dl_and_pipeline.yaml
papermill TEMPLATETARGETNAME_SLITNAME_slit_dl_and_pipeline.ipynb miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s07401_slit_dl_and_pipeline.ipynb -f miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s07401_slit_dl_and_pipeline.yaml
papermill TEMPLATETARGETNAME_SLITNAME_slit_dl_and_pipeline.ipynb miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01449_slit_dl_and_pipeline.ipynb -f miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01449_slit_dl_and_pipeline.yaml
papermill TEMPLATETARGETNAME_SLITNAME_slit_dl_and_pipeline.ipynb miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01782_slit_dl_and_pipeline.ipynb -f miscdata/jwst_outputs/NGC6791-NEWCATALOG-GAIAEDR3_s01782_slit_dl_and_pipeline.yaml
papermill TEMPLATETARGETNAME_SLITNAME_slit_dl_and_pipeline.i

## Generate runner scripts

In [9]:
runner_template = """
import os
import sys
import tqdm
import yaml
import papermill
from multiprocessing import Pool


def f(ops):
    tpath, yamlpath, outpath = ops
    if os.path.exists(outpath):
        print('skipping', outpath, 'because it already exists')
    else:
        params = yaml.load(open(yamlpath), yaml.CLoader)
        papermill.execute_notebook(tpath, outpath, params, progress_bar=False)

if __name__ == '__main__':
    outputs = {SOUTPUTS}

    npool = 1
    if len(sys.argv) > 1:
        npool = int(sys.argv[1])

    with Pool(npool) as p:
        res = list(tqdm.tqdm(p.imap(f, outputs), total=len(outputs)))
"""[1:-1]

with (outbase / 'template_runner_pipeline.py').open('w') as f:
    f.write(runner_template
              .replace('{SOUTPUTS}', repr([tuple([str(o) for o in ops]) for ops in outputs if 'pipeline' in str(ops[0])]))
           )
    print(f.name) 

with (outbase / 'template_runner_other.py').open('w') as f:
    f.write(runner_template
              .replace('{SOUTPUTS}', repr([tuple([str(o) for o in ops]) for ops in outputs if 'pipeline' not in str(ops[0])]))
           )
    print(f.name) 

miscdata/jwst_outputs/template_runner_pipeline.py
miscdata/jwst_outputs/template_runner_other.py
